# CrossSection Notebook Pre-processing

To be latest CrossSection notebook, and a combination of CrossSection_notebook_v5 and CrossSection Post 3D v4

This notebook is for pre-processing only, as it may be running for some time multiprocessing different cross section cuts

TAG 28.02.20
- Updated with possibility to extract every n'th increment
- Updated to Python 3

OBS: When running models with beams (and el type 52), search_elem_v42f_multi_test2 could possibly be used instead to get rid of bug (and possibly introduce others - so use with caution and only for information!)


TAG 24.06.20
- Updated with release of search_elem_v42
- 

## Import and functions

In [2]:
import collections
import ipywidgets
import multiprocessing as mp
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, r'C:\git\marc_tools')
import search_elem_v45f_multi as cs
import re
import pathlib
import time
from tqdm.notebook import tqdm
from IPython.display import display, clear_output, HTML
from ipywidgets import widgets, Layout

# increase width of screen
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:76% !important; }</style>"))


# Common functions for pre- and post-processing
def find_files(run_dir, *args):
    '''
    Function that finds all files with specified ending
    
    :param run_dir: Directory containing files
    :param args: Arguments which run_dir must contain
    
    :return files: Files containing arguments

    '''
    file_paths = sorted(run_dir.rglob(f'*{args[-1]}*'))
    files = [p.relative_to(run_dir) for p in file_paths]

    for arg in args[:-1]:
        files = [f for f in files if arg.lower() in str(f).lower()]

    return files


def check_complete(folder, sts_files):
    success_files = []
    
    for sts in sts_files:
        with open(os.path.join(folder, sts), 'r') as f:
            for line in f.readlines():
                if 'Job ends with exit number :    3004' in line:
                    success_files.append(sts)
    
    return success_files

def filter_list(my_list, filter_strings):
    for f in filter_strings:
        my_list = [l for l in my_list if f not in l]
    
    return my_list

def get_max_min(df, get):
    
    if get.lower() == 'max':
        val_loc = list(df.max()).index(max(df.max()))
    elif get.lower() == 'min':
        val_loc = list(df.min()).index(min(df.min()))
    else:
        return False
    
    val_name = df.columns[val_loc].split('.')[0]
    return_df = df.iloc[:, val_loc]
    
    return val_name, return_df
      
def remove_after(var, symbol):
    if symbol in var:
        var_split = var.split(symbol)
        var = symbol.join(var_split[:-1])
    return var

def rename_cols(columns):
    rename_cols = {}
    for col in columns:
        col_val = col.replace(title, '').replace('.', '')
        try:
            col_val = int(col_val) + 1
        except ValueError:
            if col_val == '':
                col_val = 1
            else:
                pass
        rename_cols[col] = col_val
    return rename_cols


def read_cs_input(input_file):
    '''
    TODO: Find out what to do if several excel sheets - currently returns last sheet
    '''
    fi = str(input_file)
    if fi.endswith('.xlsx'):
        xl = pd.ExcelFile(excel_dir)

        for sheet in xl.sheet_names:
            dfn = xl.parse(sheet)
        
        # OBS: Returns last sheet!
        return dfn
    
    elif fi.endswith('.csv'):
        # Post-process csv
        return ''
    else:
        print(f'Check that {fi} is excel- (.xlsx) or csv-file (.csv)')


ImportError: DLL load failed while importing py_post: The specified module could not be found.

In [1]:
import sys
sys.path.insert(0, r'C:\git\marc_tools')
import py_post
sys.path

ImportError: DLL load failed while importing py_post: The specified module could not be found.

## Input

In [ ]:
# Input
project = r'1978_BP_Tortue_EF_Torque\Phase1'
run = 'Analysis'

nskip = 2 # skips every nskip'th increment - Should be 0?

# Testing purposes
# project = r'FEM\FE_checks'
# run = 'Element_type_ring_stiffness'

# CS Input
dimension = '3D'

# Establishing directories
project_folder = pathlib.Path(r'D:/Projects', project)
run_folder = project_folder / run

# Calculate plastic strain ?
calcPl = 0

# Manipulate curvature calculation? default = '' (0.5)
DiamTol = '0.5'

# CrossSection
cb = elsetl = nodF = tenCB = ''
si = sym = stressT = refLc = nas = immRem = 0
txt = 1
refL1 = 2
var_list = []

# Set dimension
if dimension.lower() == 'axi':
    extp = 2 # = linear
elif dimension.lower() == '3d':
    extp = 3 # = combined
else:
    print('Dimension {} is invalid!'.format(dimension))

## Pre-processing

In [ ]:
# input
# model_name = 'Balder_2p_t' # only models containing this name
model_name = 'Full_t1'
filter_strings = ['Old'] # names that files should not contain

post_nodes = {'Full_t12' : 'full_nodes.txt',
              'Full_t13' : 'full_nodes.txt',
             } # note that these files should be in 'folder'
# {'models containing this string' : 'should use this file'}

# find all relevant files
dat_files = find_files(run_folder, model_name, '.dat')
t16_files = find_files(run_folder, model_name, '.t16')
sts_files = find_files(run_folder, model_name, '.sts')

# filter out cases containing specific strings
load_cases = [l for l in t16_files if not any(f in str(l) for f in filter_strings)]
cases = [l.parent / l.stem for l in load_cases]
dat_cases = [l for l in dat_files if not any(f in str(l) for f in filter_strings)]
dat = [l.parent / l.stem for l in dat_cases]
# cases = [l.stem for l in load_cases] NEW

# Find csv files that exists (not necessarily run)
sorted_csv = {}
csv_files = list(run_folder.rglob('*cb.csv'))

# Sort csv so that csv-files are tied to correct model
for csv_file in csv_files:
    csv_info = csv_file.name.rpartition('_')
    model, csv = csv_info[0], csv_info[2]
    
    # Check if empty
    csv_file_dir = run_folder / csv_file
    if csv_file_dir.stat().st_size != 0:
    
        if model not in sorted_csv.keys():
            sorted_csv[model] = [csv]
        else:
            sorted_csv[model].append(csv)

completed = list(sorted_csv.keys())
not_completed = [c for c in cases if c.stem not in completed]

# Creating widgets to selected load cases to run
load_cases = widgets.SelectMultiple(options = dat, value = dat, disable = False, 
                                    rows = len(cases), layout = Layout(width = '50%'))
# wg_dat = widgets.SelectMultiple(options = dat, value = dat, disable = False, 
#                                     rows = len(cases), layout = Layout(width = '50%'))
run_button = widgets.Button(description='Run CrossSection on below cases', layout=Layout(width = '50%'))
info_button = widgets.Button(description='CrossSection is completed on these cases', layout=Layout(width = '50%'))

# style
info_button.style.button_color='#23DFAC'
run_button.style.button_color='#f78585'

ui_over = widgets.HBox([run_button, info_button])
ui_under = widgets.HBox([load_cases])

def run_on_click_pre(a):
    var_list = []
#     sleep_seconds = 6*60*60
#     time.sleep(sleep_seconds)
    for case in load_cases.value:
        
        # find box tolerance
        case_file = run_folder / case.with_suffix('.sts')
        df = pd.read_csv(case_file, skiprows=9, skipfooter=4, 
                         header=None, sep='\s+', engine='python') # TODO: Fix NAN
        boxTol = max(df.values[:,-1]) + 1
        
        # make dat and .t16 file directory
        dat_file = run_folder / case.with_suffix('.dat')
        t16_file = run_folder / case.with_suffix('.t16')
        
        # find node file for the case - first match in post_node dictionary
        for key, value in post_nodes.items():
            if key.lower() in str(case).lower():
                node_file = run_folder / value
                continue
        
#         if not os.path.isfile(node_file):
#             print('Node file does not match')
        
        # create variable list for post_cases and nodes in node_file
        with node_file.open('r') as f:
            for line in f:
                # Read nodes from file
                nodes = re.findall(r'\d+', line)
                [n1, n2, n3, n4] = [int(i) for i in nodes] + ['']*(4-len(nodes))
                
                # Check if file exists - skip if it does!
                var = [str(dat_file),str(t16_file),n1,n2,n3,n4,'','',si,txt,extp,sym,stressT,elsetl,refL1,refLc,nas,tenCB,boxTol,immRem,calcPl,DiamTol,nskip]
                if case in sorted_csv.keys():
                    if str(n1) not in [re.findall(r'\d+', n)[0] for n in sorted_csv[case]]:
                        var_list.append(var)
                else:
                    var_list.append(var)
                    
#     print(var_list)
    # create pool and progress bar
    pool = mp.Pool(processes=9)
#     for _ in tqdm.tqdm_notebook(pool.imap_unordered(cs.search_elem, var_list), total=len(var_list)):
    for _ in tqdm(pool.imap_unordered(cs.search_elem, var_list), total=len(var_list)):
        pass
    pool.terminate()
    pool.join()
    
    
display(ui_over, ui_under)
run_button.on_click(run_on_click_pre)